In [ ]:
1/ Bảng phân công công việc:
Tên thành viên	MSSV	Công việc	Tỉ lệ
Lê Tâm Anh	19127330	Parse HTML để lấy thông tin Users 	33%
Trần Minh Thiện	19127281	Parse HTML để lấy thông tin Tracks & 	33%
Vương Thành An	19127326	Parse HTML để lấy thông tin Playlists 	34%

In [ ]:
import requests
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


#Tạo biến đếm a để đếm số lượng user link đã lấy được
a = 0

#Tạo list để lưu và mảng A chứa các kí tự cần tìm
link = []
A = ["a","b","c","d","e","f","g","i","l","n","o","q"]
 
for z in A:
    
#Tạo và gọi đường dẫn của chromedriver vào để chạy với thư viện selenium.
    PATH = r"chromedriver.exe"
    driver = webdriver.Chrome(PATH)
    
#Dùng hàm get() để load vào các trang tìm kiếm các user.
    driver.get("https://soundcloud.com/search/people?q=" + z)
    
    
    previous_height = driver.execute_script('return document.body.scrollHeight') 
    time.sleep(3)

#Scroll trang với số lần xác định để có thể lấy được số lượng mong muốn.    
    t = 1
    while (t < 11):
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(3)
        
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == previous_height:
            break
        previous_height = new_height
        t = t + 1

#Tìm vào nơi chứa link các user bằng class name, tag name.        
    search = driver.find_elements(By.CLASS_NAME,"sc-media-content")
    for i in range(0,len(search),1):
        title = search[i].find_elements(By.CLASS_NAME,"userItem__title")
        for i in range(0,len(title),1):
            url = title[i].find_elements(By.TAG_NAME,"a")
            for i in range(0,len(url),1):
                print(url[i].get_attribute("href"))
                link.append(url[i].get_attribute("href"))
                a = a + 1
    
    driver.quit()

print(a)
#Đưa list các user link đã tìm được vào dataframe và xuất ra file
df = pd.DataFrame(link)                  
df.to_csv('link.tsv', sep="\t",index=False)

In [30]:
import requests
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
listurl = []

#Xủ lý file chứa link của user đã lấy về để tạo ra mảng nhằm đưa vào hàm get() bên dưới.
for line in open('link.tsv'):
    line = line.replace('\n','')
    listurl.append(line)

#List chứa các giá trị tìm được để đưa vào dataframe
User_name = []
Full_name = []
From = []
Followers = []
Following = []
Tracks = []
Playlists=[]

abc=0
for i in range(len(listurl)):

#Tạo và gọi đường dẫn của chromedriver vào để chạy với thư viện selenium.
    PATH = r"chromedriver.exe"
    driver = webdriver.Chrome(PATH)
#Dùng hàm get() để load vào các trang của user.
    driver.get(listurl[i])
 
    
#Scroll trang với số lần xác định để có thể lấy được số lượng mong muốn.
    previous_height = driver.execute_script('return document.body.scrollHeight') 
    time.sleep(3)
        
    t = 1
    while t<2:
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(3)
            
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == previous_height:
            break
        previous_height = new_height
        t = t + 1
    

#User_name
#Tìm class chứa User_name sau đó lọc ra các từ không liên quan.
    user = driver.find_elements(By.CLASS_NAME,"profileHeader__info")
    list_user = user[0].text.split('\n')
    list_user[0] = list_user[0].replace(' Verified','')
    print("Username: " + list_user[0])
    User_name.append(list_user[0])
    
    
    
#Full_name   
#Tìm class chứa Full_name sau đó gọi ra.
    Fname = driver.find_elements(By.XPATH,"//h3[@class='profileHeaderInfo__additional g-type-shrinkwrap-block theme-dark g-type-shrinkwrap-large-secondary']")
    if(len(Fname) != 0):
        print("Fullname: " + Fname[0].text)
        Full_name.append(Fname[0].text)
    else:
        Full_name.append("None")


#Where
#Tìm class chứa địa chỉ user sau đó gọi ra.   
    fro = driver.find_elements(By.XPATH,"//h3[@class='profileHeaderInfo__additional g-type-shrinkwrap-block theme-dark g-type-shrinkwrap-large-secondary sc-mt-1x']")
    if(len(fro) != 0):
        From.append(fro[0].text)
        print(fro[0].text)
    else:
        From.append("None")
    
    
    
#Followers
#Tìm class chứa Followers sau đó lọc ra các từ không liên quan, chỉ giữ lại số lượng Followers.
    track = driver.find_elements(By.XPATH, "//a[@class='infoStats__statLink sc-link-light sc-link-primary']")
    if "followers" in track[0].get_attribute("title"):
        a = track[0].get_attribute("title").replace(' followers','')
        a = a.replace(',','')
        a = int(a)
        print(f'Followers: {a}')
        Followers.append(a)
    elif "follower" in track[0].get_attribute("title"):
        a = track[0].get_attribute("title").replace(' follower','')
        a = a.replace(',','')
        a = int(a)
        print(f'Followers: {a}')
        Followers.append(a)
    
#Following
#Tìm class chứa Following sau đó lọc ra các từ không liên quan, chỉ giữ lại số lượng Following.
    if "people" in track[1].get_attribute("title"):
        a = track[1].get_attribute("title").replace('Following ','')
        a = a.replace(' people','')
        a = a.replace(',','')
        a = int(a)
        print(f'Following: {a}')
        Following.append(a)
    elif "person" in track[1].get_attribute("title"):
       a = track[1].get_attribute("title").replace('Following ','')
       a = a.replace(' person','')
       a = a.replace(',','')
       a = int(a)
       print(f'Following: {a}')
       Following.append(a)
    
    
#tracks    
#Tìm class chứa số lượng Tracks sau đó lọc ra các từ không liên quan, chỉ giữ lại số lượng Tracks.
    if "tracks" in track[2].get_attribute("title"):
        a = track[2].get_attribute("title").replace(' tracks','')
        a = a.replace(',','')
        a = int(a)
        print(f'Tracks: {a}')
        Tracks.append(a)
    elif "track" in track[2].get_attribute("title"):
        a = track[2].get_attribute("title").replace(' track','')
        a = a.replace(',','')
        a = int(a)
        print(f'Tracks: {a}')
        Tracks.append(a)
#Playlists
# lấy link dẫn đến user +/sets để đến trang playlist của user
    driver.get(listurl[i]+"/sets")                        
    previous_height = driver.execute_script('return document.body.scrollHeight')
    temp=0
# cuộn xuống cho đến khi thấy được hết tất cả playlist thì lấy về
# Vì nếu không cuộn xuống thì nó sẽ không hiện ra tất cả playlist
    while True:                                            
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(1)
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == previous_height:
                break
        previous_height = new_height
        playlist_count=driver.find_elements(By.CLASS_NAME,"soundList__item")
        if temp>=len(playlist_count):
                break
        temp=len(playlist_count)
# lấy độ dài của class soundList__item sẽ ra số playlist của user
    print("Playlists: ",len(playlist_count))
    Playlists.append(len(playlist_count))                      
    driver.quit()

df = pd.DataFrame({"Username":User_name,"User_link": listurl,
                  "Fullname": Full_name,"Where": From,
                   "Followers":Followers,"Following":Following,
                   "Tracks":Tracks,"Playlists":Playlists})

#Tạo file User.csv bằng dataframe đã lập ra ở trên.
df.to_csv('user.csv', sep="\t",index=False)



<ipython-input-30-1590c9dcbc80>:31: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


Username: airnewsalerts
Fullname: All India Radio News
New Delhi, India
Followers: 21606
Following: 0
Tracks: 19096
Playlists:  13
Username: ali
Followers: 2659172
Following: 0
Tracks: 37
Playlists:  0
Username: AskPat
Fullname: Pat Flynn
San Diego, United States
Followers: 429628
Following: 1
Tracks: 23
Playlists:  8
Username: Americarnage
Fullname: Americarnage Podcast
London, England
Followers: 831719
Following: 1
Tracks: 4
Playlists:  3


In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [20]:
import requests
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from re import search
import csv
import re
import random


link = []
df = pd.read_csv('user.csv')  # load dữ liệu user từ file user.csv
new_url_playlist=[] # list để chứa link dẫn đến các playlist
for link in df.User_link:
    PATH = r"chromedriver.exe" # path đường dẫn chromedriver.exe nếu chưa có thì tải tại https://chromedriver.chromium.org/downloads
    driver = webdriver.Chrome(PATH)
    print(link)
    s2 = link.split('com/')[1] # tách ra để lấy username của user và in ra
    print(s2)
    driver.get(link+"/sets") # ghép link đã cắt với /sets để truy cập vào trang playlist của từng user
    previous_height = driver.execute_script('return document.body.scrollHeight')  # tạo biến lưu vị trí hiện tại của scroll
    time.sleep(1.5)
    t=1
    while t<5:
          driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")    
          time.sleep(1.5)
          new_height = driver.execute_script('return document.body.scrollHeight') 
                                                # cho scroll xuống thêm để hiện ra nhiếu playlist hơn
          if new_height == previous_height:     # kiểm tra nếu đã kéo hết scroll ở trang đó thì break
                 break                          # hoặc kéo không quá 4 lần để qua lấy playlist của user khác chứ không lấy hết toàn bộ của 1 user
          previous_height = new_height
          t = t + 1
    # lấy hết toàn bộ link dẫn tới các playlist của user
    link_playlist=driver.find_elements(By.CLASS_NAME,"soundTitle__usernameTitleContainer") 
    #cho vòng lặp chạy qua lần lượt các link playlist xử lý để thành link hoàng chỉnh và append vào new_url_playlist 
    # cho lấy khoảng tầm hơn 1100 link là đủ
    for j in range(0,len(link_playlist),1):
                  the_ultimate_link=link_playlist[j].find_elements(By.TAG_NAME,"a")
                 # for g in range(0,len(the_ultimate_link),1):
                  the_ultimate_link2=the_ultimate_link[1].get_attribute("href")
                  true_form = the_ultimate_link2.split('com/')[1]
                  print("true_form",true_form)
                  true_form2=true_form.split('/')[0]
                  print("true_form2",true_form2)
                  if true_form2==s2 and "/sets/" in str(the_ultimate_link2):
                     # if(the_ultimate_link2!=link[i]):
                      print("the_ultimate_link2",the_ultimate_link2)
                      new_url_playlist.append(the_ultimate_link[1].get_attribute("href"))
    print(len(new_url_playlist))
    if len(new_url_playlist)>1100:
        break
                        
                        
                        
with open("playlist.csv", "w",encoding="utf-8",newline='') as f: # tạo file playlist.csv và viết các cột thuộc tính vào file
        csv_write=csv.writer(f)
        csv_write.writerow(["playlist_url","playlist_name",
                            "user_name","track_counts","duration",
                            "like_counts","reposts_counts","date_release","tags","tracks_name"
                                    ])
        print(len(new_url_playlist))
        a = 0
        count=0 # biến count để xem đang chạy tới link thứ mấy
        for a in range(0,len(new_url_playlist)):
            count=count+1
            print(count)
            PATH = r"chromedriver.exe"
            driver = webdriver.Chrome(PATH)                
            print(new_url_playlist[a])
            driver.get(new_url_playlist[a])           # gọi webdriver mở link trong new_url_playlist lên và dò thông tin
            
            name_find2=driver.find_elements(By.CLASS_NAME,"fullHero__title")       # tìm  class chứa tên playlist   
            for i in range(0,len(name_find2),1):
                name_find3=name_find2[i].find_elements(By.TAG_NAME,"span")
            user_find=driver.find_elements(By.CLASS_NAME,"userBadge__title")      # tìm tên user
            for i in range(0,len(user_find),1):
                user_find2=user_find[i].find_elements(By.TAG_NAME,"span")
            #print(name_find3[0].get_attribute("innerHTML")) 
            try:                     
                final_name=name_find3[0].get_attribute("innerHTML")   # lấy dữ thông tin tên playlist bên trong thẻ
                final_user=user_find2[0].get_attribute("innerHTML")    # lấy dữ thông tin tên user bên trong thẻ    
            except StaleElementReferenceException as Exception:       # ở đây phải thêm try và except StaleElementReferenceException
                                                                      # để tránh bị lỗi  stale element reference exception và 
                                                                      #khi đang chạy gặp lỗi này thì chương trình sẽ bị lỗi
                                                                    # thường gặp khi get_attribute"innerHTML"
                                                                    # và khi nó dính vào lỗi này thì ở phần except 
                                                                  #chúng ta sẽ cho lấy lại tên user  và tên playlist như lúc đầu
                print('StaleElementReferenceException')
                for i in range(0,len(name_find2),1):
                    name_find3=name_find2[i].find_elements(By.TAG_NAME,"span")
                user_find=driver.find_elements(By.CLASS_NAME,"userBadge__title")
                for i in range(0,len(user_find),1):
                    user_find2=user_find[i].find_elements(By.TAG_NAME,"span")
                final_name=name_find3[0].get_attribute("innerHTML")
                final_user=user_find2[0].get_attribute("innerHTML")
            print("final_name: ",final_name)
            print("final_user: ",final_user)
# Lấy class chứa lượt likes, repost của playlist
            temp1=driver.find_elements(By.CLASS_NAME,"listenEngagement__footer")    
            for i in range(0,len(temp1),1):
                temp3=temp1[i].find_elements(By.TAG_NAME,"li")
            plays="0"
            likes="0"
            reposts="0"
# Do trong likes và repost các số quá lớn sẽ có dấu , nên ta xóa dấu,
# và dùng thư viện re để tìm số likes,repost trong chuỗi và lấy ra
            if(len(temp3)>=1):
               likes=temp3[0].get_attribute("title")
               likes=str(likes).replace(",","")
               likes=re.findall(r'\b\d+\b', likes)       
               likes=likes[0]
            if(len(temp3)>=2):
                reposts=temp3[1].get_attribute("title")
                reposts=str(reposts).replace(",","")
                reposts=re.findall(r'\b\d+\b', reposts)
                reposts=reposts[0]
            print(likes)
            print(reposts)
            get_date_2=driver.find_elements(By.CLASS_NAME,"relativeTime")   # lấy thời gian up playlist lên soundclould
            if(len(get_date_2)>0):
                    date_release=get_date_2[0].get_attribute("title")
                    print(date_release)
                
            else:
                date_release="None"
                print(date_release)
            get_track_counts=driver.find_elements(By.CLASS_NAME,"genericTrackCount") 
            
# lấy số track trong playlist
# Do dưới đấy phải lấy thuộc tính innerHTML nên phải dùng try except để tránh lỗi StaleElementReferenceException như phía trên
            if len(get_track_counts)>0:
                try:
                    get_track_counts=get_track_counts[0].find_elements(By.CLASS_NAME,"genericTrackCount__title")
                    if len(get_track_counts)>0:
                        get_track_counts=get_track_counts[0].get_attribute("innerHTML")
                    else:
                        get_track_counts=0
                except StaleElementReferenceException as Exception:
                    print('StaleElementReferenceException')
                    get_track_counts=driver.find_elements(By.CLASS_NAME,"genericTrackCount")
                    get_track_counts=get_track_counts[0].find_elements(By.CLASS_NAME,"genericTrackCount__title")
                    get_track_counts=get_track_counts[0].get_attribute("innerHTML")
                    
            else:
                get_track_counts=0
            print(get_track_counts)
# tiếp tục lấy tổng thời lượng của playlist
            if str(get_track_counts)=="0":
                get_duration="00:00"
            else:
                get_duration=driver.find_elements(By.CLASS_NAME,"genericTrackCount")   
                if len(get_duration)>0:
                    try:
                        get_duration=get_duration[0].find_elements(By.CLASS_NAME,"genericTrackCount__duration")
                        if len(get_duration)>0:
                            get_duration=get_duration[0].get_attribute("innerHTML")
                        else:
                            get_duration="00:00"
                    except StaleElementReferenceException as Exception:
                        print('StaleElementReferenceException')
                        get_duration=driver.find_elements(By.CLASS_NAME,"genericTrackCount")
                        get_duration=get_duration[0].find_elements(By.CLASS_NAME,"genericTrackCount__duration")
                        if len(get_duration)>0:
                            get_duration=get_duration[0].get_attribute("innerHTML")
                        else:
                            get_duration="00:00"
            print(get_duration)
 # lấy tag của playlist
            get_tags=driver.find_elements(By.CLASS_NAME,"soundTitle__title")         
            get_tag=driver.find_elements(By.CLASS_NAME,"fullHero__info")
           # print(get_tag)
            get_tag3="None"
            for i in range(0,len(get_tag),1):
                get_tag2=get_tag[i].find_elements(By.TAG_NAME,"a")
                #print(get_tag2)
                try:
                    for i in range(0,len(get_tag2),1):
                        get_tag_temp=get_tag2[i].get_attribute("innerHTML")
                        if(get_tag3!=""):
                            get_tag3=get_tag_temp
                            get_tag3=get_tag3.split(">")[1]
                            get_tag3=get_tag3.split("</")[0]
                            get_tag3=get_tag3.replace("&amp","")
                except StaleElementReferenceException as Exception:
                    get_tags=driver.find_elements(By.CLASS_NAME,"soundTitle__title")
                    get_tag=driver.find_elements(By.CLASS_NAME,"fullHero__info")
                   # print(get_tag)
                    get_tag3="None"
                    for i in range(0,len(get_tag),1):
                        get_tag2=get_tag[i].find_elements(By.TAG_NAME,"a")
                        #print(get_tag2)
                        for i in range(0,len(get_tag2),1):
                            get_tag_temp=get_tag2[i].get_attribute("innerHTML")
                            if(get_tag3!=""):
                                get_tag3=get_tag_temp
                                get_tag3=get_tag3.split(">")[1]
                                get_tag3=get_tag3.split("</")[0]
                                get_tag3=get_tag3.replace("&amp","")
            print(get_tag3)
            
# lấy tên toàn bộ bài hát trong playlist

            get_tracks_name=driver.find_elements(By.CLASS_NAME,"trackList__list") 
            if len(get_tracks_name)>0:
                    try:
                        get_tracks_name=get_tracks_name[0].find_elements(By.CLASS_NAME,"trackList__item")
                        tracks_name=""
                        for i in range(0,len(get_tracks_name)):
                            get_track_name_next=get_tracks_name[i].find_elements(By.CLASS_NAME,"trackItem")
                            get_track_name_next=get_track_name_next[0].find_elements(By.CLASS_NAME,"trackItem__content")
                            track_name=get_track_name_next[0].find_elements(By.CLASS_NAME,"trackItem__trackTitle")
                            if len(track_name)>0: 
                                track_name=track_name[0].get_attribute("innerHTML")
                            else:
                                track_name="None"
                            tracks_name+=str(track_name)+","
                        tracks_name=tracks_name[:-1]
                    except StaleElementReferenceException as Exception:
                        get_tracks_name=driver.find_elements(By.CLASS_NAME,"trackList__list")
                        get_tracks_name=get_tracks_name[0].find_elements(By.CLASS_NAME,"trackList__item")
                        tracks_name=""
                        for i in range(0,len(get_tracks_name)):
                            get_track_name_next=get_tracks_name[i].find_elements(By.CLASS_NAME,"trackItem")
                            get_track_name_next=get_track_name_next[0].find_elements(By.CLASS_NAME,"trackItem__content")
                            track_name=get_track_name_next[0].find_elements(By.CLASS_NAME,"trackItem__trackTitle")
                            if len(track_name)>0: 
                                track_name=track_name[0].get_attribute("innerHTML")
                            else:
                                track_name="None"
                            tracks_name+=+str(track_name)+","
                        tracks_name=tracks_name[:-1]
            else:
                tracks_name="None"
            print(tracks_name)      
# viết các thuộc tính vào đúng cột theo quy định
            csv_write.writerow([new_url_playlist[a], 
                                 final_name,final_user,get_track_counts,get_duration,
                                 likes,reposts,date_release,get_tag3,tracks_name])

driver.quit()
driver.quit()                         
driver.quit()                    



<ipython-input-20-38f8c6903bdf>:31: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


https://soundcloud.com/allindiaradionews
allindiaradionews
true_form allindiaradionews/sets/bulletins
true_form2 allindiaradionews
the_ultimate_link2 https://soundcloud.com/allindiaradionews/sets/bulletins
true_form allindiaradionews/sets/pm-mann-ki-baat
true_form2 allindiaradionews
the_ultimate_link2 https://soundcloud.com/allindiaradionews/sets/pm-mann-ki-baat
true_form allindiaradionews/sets/recordings
true_form2 allindiaradionews
the_ultimate_link2 https://soundcloud.com/allindiaradionews/sets/recordings
true_form allindiaradionews/sets/interviews
true_form2 allindiaradionews
the_ultimate_link2 https://soundcloud.com/allindiaradionews/sets/interviews
true_form allindiaradionews/sets/programs
true_form2 allindiaradionews
the_ultimate_link2 https://soundcloud.com/allindiaradionews/sets/programs
true_form allindiaradionews/sets/samayiki
true_form2 allindiaradionews
the_ultimate_link2 https://soundcloud.com/allindiaradionews/sets/samayiki
true_form allindiaradionews/sets/spotlight
true

48
https://soundcloud.com/ammartv
ammartv
true_form ammartv/sets/muhammad-taufik
true_form2 ammartv
the_ultimate_link2 https://soundcloud.com/ammartv/sets/muhammad-taufik
true_form ammartv/sets/misyari-rasyid
true_form2 ammartv
the_ultimate_link2 https://soundcloud.com/ammartv/sets/misyari-rasyid
true_form ammartv/sets/muhammad-aulia-putra
true_form2 ammartv
the_ultimate_link2 https://soundcloud.com/ammartv/sets/muhammad-aulia-putra
true_form ammartv/sets/muchlash-as-sakalany-quran
true_form2 ammartv
the_ultimate_link2 https://soundcloud.com/ammartv/sets/muchlash-as-sakalany-quran
true_form ammartv/sets/lutfi-maricar-quran-reciter
true_form2 ammartv
the_ultimate_link2 https://soundcloud.com/ammartv/sets/lutfi-maricar-quran-reciter
true_form ammartv/sets/al-hikam-sabirullah-quran
true_form2 ammartv
the_ultimate_link2 https://soundcloud.com/ammartv/sets/al-hikam-sabirullah-quran
true_form ammartv/sets/syaikh-abdurahman-al-ausy
true_form2 ammartv
the_ultimate_link2 https://soundcloud.com/

true_form avanoticias-xalapa/sets/14-03-21-2
true_form2 avanoticias-xalapa
the_ultimate_link2 https://soundcloud.com/avanoticias-xalapa/sets/14-03-21-2
true_form avanoticias-xalapa/sets/14-03-21-1
true_form2 avanoticias-xalapa
the_ultimate_link2 https://soundcloud.com/avanoticias-xalapa/sets/14-03-21-1
true_form avanoticias-xalapa/sets/14-03-21
true_form2 avanoticias-xalapa
the_ultimate_link2 https://soundcloud.com/avanoticias-xalapa/sets/14-03-21
true_form angelothekid/playpen-episode-1
true_form2 angelothekid
true_form angelothekid/live-sessions-episode-1-angelo-the-kid
true_form2 angelothekid
106
https://soundcloud.com/alexhorne
alexhorne
true_form alexhorne/sets/dot-dot-dot
true_form2 alexhorne
the_ultimate_link2 https://soundcloud.com/alexhorne/sets/dot-dot-dot
true_form alexhorne/sets/alex-horne-breaks-the-news
true_form2 alexhorne
the_ultimate_link2 https://soundcloud.com/alexhorne/sets/alex-horne-breaks-the-news
108
https://soundcloud.com/atw9k
atw9k
true_form atw9k/sets/tourna

https://soundcloud.com/arianagrandeforrealidkwhyicantgetmyownlinkbutok
arianagrandeforrealidkwhyicantgetmyownlinkbutok
155
https://soundcloud.com/audiosach
audiosach
true_form audiosach/sets/t-i-th-y-hoa-v-ng-tr-n-c-xanh
true_form2 audiosach
the_ultimate_link2 https://soundcloud.com/audiosach/sets/t-i-th-y-hoa-v-ng-tr-n-c-xanh
true_form audiosach/sets/l-n-m-trong-l
true_form2 audiosach
the_ultimate_link2 https://soundcloud.com/audiosach/sets/l-n-m-trong-l
true_form audiosach/sets/ng-i-kh-c-tr-n-c-y
true_form2 audiosach
the_ultimate_link2 https://soundcloud.com/audiosach/sets/ng-i-kh-c-tr-n-c-y
true_form audiosach/sets/cafe-c-ng-tony-tony-bu-i-s-ng
true_form2 audiosach
the_ultimate_link2 https://soundcloud.com/audiosach/sets/cafe-c-ng-tony-tony-bu-i-s-ng
true_form audiosach/sets/ch-ng-trai-kh-ng-bi-t-y-u
true_form2 audiosach
the_ultimate_link2 https://soundcloud.com/audiosach/sets/ch-ng-trai-kh-ng-bi-t-y-u
true_form audiosach/sets/y-u-nh-m-ch-hai-c-nh-m-em-g-1
true_form2 audiosach
the_u

...

https://soundcloud.com/agora-philosophical-forum/sets/historyoffreedom
final_name:  فصل آزادی
final_user:  Agora Philosophical Forum
76
7
4 November 2013
8
9:19:39
فلسفه
None_تاریخ آزادی - نشست یکم: آزادی چیست؟,None_تاریخ آزادی - نشست دوم: آزادی مثبت و آزادی منفی,None_تاریخ آزادی - نشست سوم: اندیشۀ آزادی و انقلاب مشروطه در ایران,None_تاریخ آزادی - نشست چهارم: روسو یا کنستان؟ آزادی نزد قدما و متجددین,None_تاریخ آزادی - نشست ششم: بازخوانی روسو در ایران؛ آزادی‌های مدنی یا اقتدارگرایی مردمی؟,None_تاریخ آزادی - نشست هفتم: آیا آزادی بدون اخلاق ممکن است؟,None_تاریخ آزادی - نشست هشتم: آزادی فردی یا آزادی جمعی؛ تحلیلی از رمان و فیلم سرچشمه,None_تاریخ آزادی - نشست نهم: آزادی و جامعه مدنی
1028
https://soundcloud.com/abracadabra/sets/amine-k-taran-abra023-clips
final_name:  Amine K - Taran (ABRA023) [clips]
final_user:  Abracadabra
9
1
4 October 2021
2
4:00
Deep House
None_Amine K - Taran (El Loko Mix) [clip],None_Amine K - Taran (Dr Gonzo Mix) [clip]
1029
https://soundcloud.com/abracadabra/sets/f

https://soundcloud.com/abracadabra/sets/blondish-x-rowee-garden-of-3den-abra015-clips
final_name:  BLOND:ISH X ROWEE Garden Of 3Den (ABRA015) [clips]
final_user:  Abracadabra
49
11
1 December 2020
2
4:00
Deep House
None_BLOND ISH X ROWEE - Garden Of 3Den (ABRA015) [clip],None_BLOND ISH X ROWEE - Garden Of 3Den (Meditation Mix) (ABRA015) [clip]
1048
https://soundcloud.com/abracadabra/sets/abracadabra-festival-1-0
final_name:  Abracadabra Festival 1.0
final_user:  Abracadabra
17
7
16 November 2020
12
16:18:33
None
None_Carlita @ Abracadabra Festival 1.0,None_Birds Of Mind @ Abracadabra Festival 1.0,None_Themba @ Abracadabra Festival 1.0,None_Jenia Tarsol @ Abracadabra Virtual Festival 1.0,None_Diplo @ Abracadabra Virtual Festival 1.0,None_Chambord @ Abracadabra Virtual Festival 1.0,None_Amine K @ Abracadabra Virtual Festival 1.0,None_Ameme @ Abracadabra Virtual Festival 1.0,None_Lunar Disco @ Abracadabra Festival 1.0,None_Pablo Fierro @ Abracadabra Festival 1.0,None_Mother Of The Sun @ A

https://soundcloud.com/user-218452092/sets/huncho-jack-like-beat-vol-2
final_name:  huncho jack like beat vol 2
final_user:  adediablo
1
1
12 September 2021
3
8:42
None
None_She Hid All Along,None_She - Day - Break - Pleasant,None_Coveting Leading On Supporting To Destroy Entertainer By Law And Financially
1069
https://soundcloud.com/user-218452092/sets/no-knowledge-gospel
final_name:  no knowledge gospel
final_user:  adediablo
0
0
23 August 2021
22
52:23
None
None_Turn It Up As You Turn Them On,None_We Been Going Like Competition Growing - 16:12:2020, 16,None_Just As Anybody Anywhere - 16:12:2020, 10,None_Cant Get Along Anymore,None_Ruined Gospelist With A  Past Prosecutable,None_1stuck In Between Old And Young And Skipped For The Younger It's Over Well Done Further,None_You Forgot,None_Up's And Downs,None_Unleess With You,None_Chill Urban Grime,None_If You're Not Here,None_The Levarage Or Advantage,None_As Said,None_Way It Is Do You Think,None_I Guess
1070
https://soundcloud.com/user

None_You Dig It Shorty,None_Too Much,None_Can No One Do It Yet,None_Layy Out The Routines For Deuces,None_When Your Haters Strike With Court Verdict At Hand,None_Tag And Smoke Like Ash,None_Tab To Add,None_When They Remoted Personel Successfully Then Lie Detector,None_Who Did You Sleep With To Block Others,None_Goku Factor N Dubstep N Trap
1085
https://soundcloud.com/user-218452092/sets/epic-living-vol34-with-c4
final_name:  epic living vol34 with c4
final_user:  adediablo
0
0
10 June 2021
10
33:38
None
None_Confirming Fears Off,None_Preparing For No Cash,None_Bradley And His,None_Espionage Prostitutes And Polite Ruins,None_Who Is To You Or Me,None_Song For Who Just Your Love,None_Celebrating While Delibarating Foes Futures The Ones Etc,None_They Caused Rainfall For Twenty Seventy Years U.k Women In Beh,None_Any,None_Turn Agaisnt Anyone
1086
https://soundcloud.com/user-218452092/sets/grime-4
final_name:  grime 4
final_user:  adediablo
0
0
8 June 2021
4
12:26
None
None_Hungry Cracheads,

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=95.0.4638.69)
Stacktrace:
Backtrace:
	Ordinal0 [0x002E06F3+2492147]
	Ordinal0 [0x00279BD1+2071505]
	Ordinal0 [0x00182478+1057912]
	Ordinal0 [0x001758A4+1005732]
	Ordinal0 [0x001754E6+1004774]
	Ordinal0 [0x00174BB8+1002424]
	Ordinal0 [0x00173D4D+998733]
	Ordinal0 [0x00174117+999703]
	Ordinal0 [0x0017ED91+1043857]
	Ordinal0 [0x001752B2+1004210]
	Ordinal0 [0x00175EB2+1007282]
	Ordinal0 [0x001754FA+1004794]
	Ordinal0 [0x00174BB8+1002424]
	Ordinal0 [0x00173D4D+998733]
	Ordinal0 [0x00174006+999430]
	Ordinal0 [0x00183A4A+1063498]
	Ordinal0 [0x001D54BD+1397949]
	Ordinal0 [0x001C57C3+1333187]
	Ordinal0 [0x001D4F38+1396536]
	Ordinal0 [0x001C568B+1332875]
	Ordinal0 [0x001A21D4+1188308]
	Ordinal0 [0x001A302F+1191983]
	GetHandleVerifier [0x004667A6+1545030]
	GetHandleVerifier [0x0051105C+2243580]
	GetHandleVerifier [0x0036BC97+518199]
	GetHandleVer

In [9]:
import requests
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from re import search
import csv



a = 0
link = []

df = pd.read_csv('user.csv') 
users_many_Tracks=df.loc[df.Tracks>=200]
new_url_track=[]
for link in users_many_Tracks.User_link:
    PATH = r"chromedriver.exe"
    driver = webdriver.Chrome(PATH)
    print(link)
    s2 = link.split('com/')[1]
    print(s2)
    driver.get(link+"/tracks")
    previous_height = driver.execute_script('return document.body.scrollHeight') 
    time.sleep(3)
    t=1
    while t<8:
          driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
          time.sleep(3)
          new_height = driver.execute_script('return document.body.scrollHeight')
          if new_height == previous_height:
                 break
          previous_height = new_height
          t = t + 1
    link_track=driver.find_elements(By.CLASS_NAME,"soundTitle__usernameTitleContainer")
    
# =============================================================================
#    -	Sử dụng webdriver và thư viện selenium để vào các link user có số lượng track hơn 200 trong file user1.csv
# -	Sử dụng vòng lặp while để lướt trang xuống lấy các document trong trang mỗi khi kéo xuống (ở đây kéo xuống khoảng 8 lần nếu như mà trang không đủ 8 lần kéo xuống thì break)
# -	Link_track để chứa các class chứa link track   
# =============================================================================

    for j in range(0,len(link_track),1):
                  the_ultimate_link=link_track[j].find_elements(By.TAG_NAME,"a")
                 # for g in range(0,len(the_ultimate_link),1):
                  the_ultimate_link2=the_ultimate_link[1].get_attribute("href")
                  true_form = the_ultimate_link2.split('com/')[1]
                  true_form2=true_form.split('/')[0]
                  print(true_form2)
                  if(true_form2==s2):
                     # if(the_ultimate_link2!=link[i]):
                      print(the_ultimate_link2)
                      new_url_track.append(the_ultimate_link[1].get_attribute("href"))
    if(len(new_url_track)>=1000):
        break
driver.quit()
                         
# =============================================================================
# - Cho vòng lặp chạy để tiến hành láy các TAG có tên “a” từ đó lấy các thuộc tính href sẽ ra được link track
# -	Dòng code 51 đến 57 dùng để loại bỏ các track không phải của user (related track)
# -	Nếu như đủ hoặc lớn hơn 1000 track thì dừng
# -	Driver.quit() dùng để đóng trình duyệt sau khi lấy xong 
# =============================================================================
            
               

with open("track.csv", "w",encoding="utf-8",newline='') as f:
        csv_write=csv.writer(f)
        csv_write.writerow(["track_url","track_name",
                            "user_name","comment_counts",
                            "play_counts","like_counts","reposts_counts","date_release",
                             "descript","related","duration","tag"])
        print(len(new_url_track))
        for i in range(len(new_url_track)):
            if(i>0):
                print("\n")
            PATH = r"chromedriver.exe"
            driver = webdriver.Chrome(PATH)
            print(new_url_track[i])
            driver.get(new_url_track[i])
          
            name_find2=driver.find_elements(By.CLASS_NAME,"fullHero__title")
           
            for i in range(0,len(name_find2),1):
                name_find3=name_find2[i].find_elements(By.TAG_NAME,"span")
            final_name=name_find3[0].get_attribute("innerHTML")
            print(final_name)
            user_find=driver.find_elements(By.CLASS_NAME,"userBadge__title")
            for i in range(0,len(user_find),1):
                user_find2=user_find[i].find_elements(By.TAG_NAME,"span")
           
            
            final_user=user_find2[0].get_attribute("innerHTML")
            print(final_user)
# =============================================================================
# -   Tiếp theo mở file CSV và tạo ra các cột bao gồm track url, tên của track, tên của user….
# -	Cũng giống bước trên chúng ta sử dụng web driver để vào link track vừa lấy được trong list new_url_track để tiến hành lấy thông tin của track sử dụng vòng lặp chạy đến hết list (new_url_track) 
# -	Đoạn code từ 86 tới 91 là để lấy thông tin tên của track
# -	Đoạn code từ 92 đến 97 để lấy tên user của track 
# =============================================================================

            comment_find=driver.find_elements(By.CLASS_NAME,"listenDetails")
            comment_count=""
            for i in range(0,len(comment_find),1):
                comment_find2=comment_find[i].find_elements(By.TAG_NAME,"h3")
                for i in range(0,len(comment_find2),1):
                    comment_find3=comment_find2[i].find_elements(By.TAG_NAME,"span")
            if(len(comment_find3)>1):
               
               cc_final=comment_find3[1].get_attribute("innerHTML")
               cc_final=cc_final.split(' ')[0]
               print(cc_final)
               
            else:
                
                cc_final="0"
                print(cc_final)
            temp1=driver.find_elements(By.CLASS_NAME,"listenEngagement__footer")
            plays="0"
            likes="0"
            reposts="0"
            for i in range(0,len(temp1),1):
                temp3=temp1[i].find_elements(By.TAG_NAME,"li")
                for i in range(0,len(temp3),1):
                   if "plays" in temp3[i].get_attribute("title"):
                       plays=temp3[i].get_attribute("title")
                       plays=plays.split(' ')[0]
                   if "likes" in temp3[i].get_attribute("title"):
                       likes=temp3[i].get_attribute("title")
                       likes=likes.split(' ')[0]
                   if "reposts" in temp3[i].get_attribute("title"):
                       reposts=temp3[i].get_attribute("title")
                       reposts=reposts.split(' ')[0]
            print(plays)
            print(likes)
            print(reposts)
# =============================================================================
# - Đoạn code từ 106 đến 121 dùng để tìm số lượng comment của track, nếu như track có comment thì sẽ lưu số lượng comment của track,nếu không có thì để “0”
# -	Đoạn code từ 122 đến 137 dùng để lấy số lượng plays, likes , reposts của track , nếu như track không có plays hoặc likes hoặc reposts thì để “0” 
# =============================================================================

            get_date_2=driver.find_elements(By.CLASS_NAME,"relativeTime")
            if(len(get_date_2)>0):
                
                    date_release=get_date_2[0].get_attribute("title")
                    print(date_release)
            else:
                date_release="None"
                print(date_release)
            get_de=driver.find_elements(By.CLASS_NAME,"listenDetails__partialInfo")
            for i in range (0,len(get_de),1):
                get_de2=get_de[i].find_elements(By.TAG_NAME,"P")
            if(len(get_de2)>1):
                    get_de_final=get_de2[0].get_attribute("innerHTML")
                    get_de_final=get_de_final.replace("&nbsp;","")
                    print(get_de_final)
            else:
                get_de_final="None"
                print(get_de_final)
            get_related=driver.find_elements(By.CLASS_NAME,"soundTitle__titleContainer")
            get_related_final2=""
            if(len(get_related) >=2):
                get_related2=get_related[1].find_elements(By.TAG_NAME,"a")
                get_related_final=get_related2[1].get_attribute("title")
                get_related_final2=get_related_final
                print(get_related_final)
                if(len(get_related)>=3):
                    get_related2=get_related[2].find_elements(By.TAG_NAME,"a")
                    get_related_final=get_related2[1].get_attribute("title")
                    get_related_final2=get_related_final2+","
                    get_related_final2=get_related_final2+get_related_final
                    print(get_related_final)
                    if(len(get_related)>=4):
                        get_related2=get_related[3].find_elements(By.TAG_NAME,"a")
                        get_related_final=get_related2[1].get_attribute("title")
                        get_related_final2=get_related_final2+","
                        get_related_final2=get_related_final2+get_related_final
                        print(get_related_final)
            else:  
                get_related_final="None"
                print(get_related_final)
# =============================================================================
# -   Đoạn code từ 146 đến 163 dùng để lấy chú thích thêm của track, nếu track không có chú thích thêm thì lưu “None”
# -	Đoạn code từ 164 đến 185 dùng để lấy các track liên quan(related) của track, nếu track không có track nào liên quan thì lưu “None” 
# =============================================================================

            
            get_duration3=""
            driver.implicitly_wait(3)
            get_duration=driver.find_elements(By.CLASS_NAME,"playbackTimeline__duration")
            for i in range(0,len(get_duration),1):
                get_duration2=get_duration[i].find_elements(By.TAG_NAME,"span")
               
                for i in range(0,len(get_duration2),1):
                    #if(i ==1):
                            get_duration3=get_duration2[i].get_attribute("innerHTML")
                            if (i==1):
                              print(get_duration3)
            get_tag=driver.find_elements(By.CLASS_NAME,"fullHero__info")
          
            get_tag3="None"
            for i in range(0,len(get_tag),1):
                get_tag2=get_tag[i].find_elements(By.TAG_NAME,"a")
                #print(get_tag2)
                for i in range(0,len(get_tag2),1):
                    get_tag_temp=get_tag2[i].get_attribute("innerHTML")
                    if(get_tag3!=""):
                        get_tag3=get_tag_temp
                        get_tag3=get_tag3.split(">")[1]
                        get_tag3=get_tag3.split("</")[0]
                        get_tag3=get_tag3.replace("&amp","")
                        print(get_tag3)
                    else:
                        print(get_tag3)
            csv_write.writerow([new_url_track[i], 
                                  final_name,final_user,cc_final
                                  ,plays,
                                  likes,reposts,date_release,get_de_final,get_related_final2,get_duration3,get_tag3])
driver.quit()
# =============================================================================
# -	Đoạn code từ 192 đến 202 để lấy thời lượng của track 
# -	Đoạn code từ 203 đến 218 dùng để lấy tag của track (nếu có) , nếu không có thì lưu “None” 
# -	Cuối cùng là lưu vào file CSV các thông tin vừa lấy được và tiếp tục vòng lặp cho đến khi lấy được hết thông tin của track
# -	Driver.quit() để thoát trình duyệt sau khi lấy xong 
# 
# =============================================================================

<ipython-input-9-0e997adf5003>:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
1
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
2
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
3
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
4
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
5
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
6
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
7
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
8
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
9
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
10
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
11
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
12
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
13
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
14
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
15
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
16
final_name:  PM - Mann Ki Baat
fi

final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
135
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
136
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
137
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
138
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
139
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
140
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
141
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
142
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
143
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
144
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
145
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
146
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
147
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
148
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
149
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
150
final_na

final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
267
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
268
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
269
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
270
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
271
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
272
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
273
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
274
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
275
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
276
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
277
StaleElementReferenceException
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
278
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
279
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
280
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
281
final_name:  PM - Mann Ki Baat
final_us

final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
399
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
400
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
401
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
402
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
403
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
404
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
405
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
406
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
407
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
408
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
409
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
410
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
411
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
412
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
413
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
414
StaleEle

final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
531
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
532
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
533
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
534
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
535
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
536
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
537
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
538
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
539
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
540
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
541
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
542
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
543
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
544
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
545
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
546
final_na

final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
662
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
663
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
664
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
665
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
666
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
667
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
668
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
669
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
670
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
671
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
672
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
673
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
674
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
675
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
676
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
677
final_na

final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
795
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
796
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
797
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
798
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
799
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
800
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
801
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
802
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
803
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
804
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
805
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
806
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
807
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
808
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
809
final_name:  PM - Mann Ki Baat
final_user:  airnewsalerts
810
final_na

WebDriverException: Message: chrome not reachable
  (Session info: chrome=95.0.4638.69)
Stacktrace:
Backtrace:
	Ordinal0 [0x002E06F3+2492147]
	Ordinal0 [0x00279BD1+2071505]
	Ordinal0 [0x00182340+1057600]
	Ordinal0 [0x00177474+1012852]
	Ordinal0 [0x00177C76+1014902]
	Ordinal0 [0x00179345+1020741]
	Ordinal0 [0x00173453+996435]
	Ordinal0 [0x00183690+1062544]
	Ordinal0 [0x001D46D2+1394386]
	Ordinal0 [0x001C568B+1332875]
	Ordinal0 [0x001A21D4+1188308]
	Ordinal0 [0x001A302F+1191983]
	GetHandleVerifier [0x004667A6+1545030]
	GetHandleVerifier [0x0051105C+2243580]
	GetHandleVerifier [0x0036BC97+518199]
	GetHandleVerifier [0x0036AD80+514336]
	Ordinal0 [0x0027ED2D+2092333]
	Ordinal0 [0x00282EE8+2109160]
	Ordinal0 [0x00283022+2109474]
	Ordinal0 [0x0028CB71+2149233]
	BaseThreadInitThunk [0x756DFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77797A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77797A6E+238]
	(No symbol) [0x00000000]


In [37]:
import pandas as pd
df = pd.read_csv('user.csv',error_bad_lines=False) 
print(df.mean())
print(df.median())
print(df.info())

Followers    71074.178821
Following       97.249750
Tracks        1266.348651
Playlists       18.686314
dtype: float64
Followers    3764.0
Following       4.0
Tracks        112.0
Playlists       3.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1001 entries, 0 to 1000
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Username   1001 non-null   object
 1   User_link  1001 non-null   object
 2   Fullname   1001 non-null   object
 3   Where      1001 non-null   object
 4   Followers  1001 non-null   int64 
 5   Following  1001 non-null   int64 
 6   Tracks     1001 non-null   int64 
 7   Playlists  1001 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 62.7+ KB
None


In [38]:
import pandas as pd
df = pd.read_csv('track.csv') 
print(df.mean())
print(df.median())
print(df.info())

comment_counts       2.380811
play_counts       1467.719090
like_counts         42.053412
reposts_counts       5.721068
dtype: float64
comment_counts     0.0
play_counts       25.0
like_counts        0.0
reposts_counts     0.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011 entries, 0 to 1010
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   track_url       1011 non-null   object
 1   track_name      1011 non-null   object
 2   user_name       1011 non-null   object
 3   comment_counts  1011 non-null   int64 
 4   play_counts     1011 non-null   int64 
 5   like_counts     1011 non-null   int64 
 6   reposts_counts  1011 non-null   int64 
 7   date_release    1011 non-null   object
 8   descript        1011 non-null   object
 9   related         892 non-null    object
 10  duration        1011 non-null   object
 11  tag             1011 non-null   object
dtypes: int64(4), object(8)
memory 

In [39]:
import pandas as pd
df = pd.read_csv('playlist.csv') 
print(df.mean())
print(df.median())
print(df.info())

track_counts      10.784936
like_counts       30.654265
reposts_counts     4.791289
dtype: float64
track_counts      3.0
like_counts       1.0
reposts_counts    0.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1102 entries, 0 to 1101
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   playlist_url        1102 non-null   object
 1   playlist_name       1102 non-null   object
 2   user_name           1102 non-null   object
 3   track_counts        1102 non-null   int64 
 4   duration            1102 non-null   object
 5   like_counts         1102 non-null   int64 
 6   reposts_counts      1102 non-null   int64 
 7   date_release        1102 non-null   object
 8   tags                1102 non-null   object
 9   tracks_author_name  1102 non-null   object
dtypes: int64(3), object(7)
memory usage: 86.2+ KB
None
